In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.ChatCompletion;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1

## 打造 Chat 的感覺

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
Assistant:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

ChatHistory history = [];

In [4]:
var request = "Hi, I'm looking for book suggestions";
var bot_answer = await chatFunction.InvokeAsync(kernel, new (){
                    { "userInput", request },
                    { "history", string.Join("\n", history.Select(x => x.Role + ": " + x.Content)) }
                    });
                    
history.AddUserMessage(request);
history.AddAssistantMessage(bot_answer.ToString());

In [5]:
Console.WriteLine(string.Join("\n", history.Select(x => x.Role + ": " + x.Content)));

user: Hi, I'm looking for book suggestions
assistant: Sure, what kind of books are you interested in? Fiction, non-fiction, mystery, romance, science fiction, fantasy, self-help, or something else?


In [8]:
Func<string, Task> Chat = async (string input) => {

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, new (){
                    { "userInput", input },
                    { "history", string.Join("\n", history.Select(x => x.Role + ": " + x.Content)) }
                    });

history.AddUserMessage(input);
history.AddAssistantMessage(answer.ToString());
    
    // Show the response
    Console.WriteLine($"\nUser: {input}\nAssistant: {answer}\n" );
};

In [9]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");


User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
Assistant: I recommend "The Greeks" by H.D.F. Kitto. It provides a comprehensive overview of ancient Greek history, culture, and society.



In [10]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");



User: that sounds interesting, what are some of the topics I will learn about?
Assistant: You will learn about the rise of ancient Greek civilization, the development of Greek city-states, the Persian Wars, the Golden Age of Athens, Greek philosophy, art, and literature, as well as the conquests of Alexander the Great.



In [11]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");



User: Which topic from the ones you listed do you think most people find interesting?
Assistant: I think many people find the Golden Age of Athens to be particularly fascinating. It was a time of great cultural and intellectual achievement, with the flourishing of philosophy, drama, art, and architecture.



In [12]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");



User: could you list some more books I could read about the topic(s) you mentioned?
Assistant: Certainly! Here are some more books you could consider:

1. "The Peloponnesian War" by Donald Kagan - This book provides a detailed account of the conflict between Athens and Sparta, which marked the end of the Golden Age of Athens.
2. "The Birth of Classical Europe: A History from Troy to Augustine" by Simon Price and Peter Thonemann - This book offers a broader perspective on the development of ancient Greek civilization and its impact on the wider Mediterranean world.
3. "The Histories" by Herodotus - This ancient work is considered one of the earliest examples of historical writing and provides valuable insights into the Persian Wars and other events in ancient Greece.

I hope you find these suggestions helpful!



In [13]:
Console.WriteLine(string.Join("\n", history.Select(x => x.Role + ": " + x.Content)));

user: Hi, I'm looking for book suggestions
assistant: Sure, what kind of books are you interested in? Fiction, non-fiction, mystery, romance, science fiction, fantasy, self-help, or something else?
user: I would like a non-fiction book suggestion about Greece history. Please only list one book.
assistant: I recommend "The Greeks" by H.D.F. Kitto. It provides a comprehensive overview of ancient Greek history, culture, and society.
user: that sounds interesting, what are some of the topics I will learn about?
assistant: You will learn about the rise of ancient Greek civilization, the development of Greek city-states, the Persian Wars, the Golden Age of Athens, Greek philosophy, art, and literature, as well as the conquests of Alexander the Great.
user: Which topic from the ones you listed do you think most people find interesting?
assistant: I think many people find the Golden Age of Athens to be particularly fascinating. It was a time of great cultural and intellectual achievement, with

## 用迴圈方式執行

In [14]:
// Create a Semantic Kernel template for chat
var chat = kernel.CreateFunctionFromPrompt(
    @"{{$history}}
    User: {{$request}}
    Assistant: "
);

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

// Create chat history
ChatHistory history = [];

// Start the chat loop
while (true)
{
    // Get user input
    Console.Write("User > ");
    var request = await InteractiveKernel.GetInputAsync("");
    Console.Write($"\n{request}\n");

    if (request == "break")
    {
        

        break;
    }

    // Get chat response
    var chatResult = kernel.InvokeStreamingAsync<StreamingChatMessageContent>(
        chat,
        new() {
            { "request", request },
            { "history", string.Join("\n", history.Select(x => x.Role + ": " + x.Content)) }
        }
    );

    // Stream the response
    string message = "";
    await foreach (var chunk in chatResult)
    {
        if (chunk.Role.HasValue) Console.Write(chunk.Role + " > ");
        message += chunk;
        Console.Write(chunk);
    }
    Console.WriteLine();

    // Append to history
    history.AddUserMessage(request!);
    history.AddAssistantMessage(message);
}

User > 
可以推薦我東方武俠小說嗎?
assistant > 當然可以！以下是一些推薦的東方武俠小說：

1. 金庸的作品：《射雕英雄傳》、《神鵰俠侶》、《倚天屠龍記》等，金庸是中國最知名的武俠小說作家，他的作品融合了武俠、歷史與浪漫元素。

2. 古龍的作品：《絕代雙雄》、《飛狐外傳》、《雙手亂舞》等，古龍的作品以輕快的筆觸和詼諧風趣的對白著稱，是許多讀者傾心的選擇。

3. 梁羽生的作品：《白髮魔女傳》、《笑傲江湖》、《書劍恩仇錄》等，梁羽生的作品以情感豐富和劇情跌宕起伏而聞名。

這些都是非常經典的東方武俠小說，你可以根據自己的喜好選擇閱讀。希望你會找到喜歡的作品！
User > 
請給我白髮魔女傳
assistant > 《白髮魔女傳》是一部由梁羽生所著的經典武俠小說，描寫了一段充滿愛恨情仇的故事。故事的主角是白髮魔女王語嫣和她的愛人「一劍」楊蓮亭，他們的愛情經歷了種種考驗。王語嫣因為服用了毒藥而變得神秘、冷酷和充滿魔性，但她的愛情卻依然動人。這部小說融合了武俠、愛情和江湖義氣等元素，深受讀者喜愛。希望你會喜歡這個故事！
User > 
你不要亂講，王語嫣和楊蓮亭    一個是天龍八部  一個是笑傲江湖
assistant > 抱歉，我的描述有些錯誤。《白髮魔女傳》是梁羽生的作品，它講述了白髮魔女王語嫣和她的愛人「風清扬」之間的故事。這個故事充滿了愛恨情仇和武林江湖的糾葛，情節跌宕起伏，絕對值得一讀。我混淆了角色名字，謹此致歉。希望您能夠喜歡這個故事！
User > 
玉羅剎
assistant > 《玉羅剎》是梁羽生所著的武俠小說，講述了一段充滿江湖風雲和感人愛情的故事。故事的主角是石中玉和藍秀儀，兩人情深意切，但卻被種種陰謀和宿命所纏繞。這部小說以其精彩的情節和鮮明的人物形象贏得了廣大讀者的喜愛。希望你會喜歡這個故事！
User > 
藍秀儀是出自哪本小說?
assistant > 抱歉，藍秀儀是出自古龍的作品《楚留香》系列中的角色，而非梁羽生的作品。謹此致歉。
User > 
卓一航才根白髮魔女傳有關係
assistant > 卓一航是《白髮魔女傳》中的另一個重要角色，他和王語嫣之間有一段復雜的情感糾葛，並深深地受到王語嫣的影響。這使得《白髮魔女傳》中的人物關係更為錯綜複雜，為故事的發展增添了許多劇情變化。希望你會喜歡這個充滿愛恨情仇的故事！
U